In [17]:
import os
import pandas as pd
from ipyevents import Event as Event
from ipycanvas import Canvas, MultiCanvas, hold_canvas
import PIL
from PIL import ImageOps
from ipywidgets import Label, HTML, HBox, Image, VBox, Box, HBox, ColorPicker, IntSlider, link, AppLayout, IntProgress, Button
from IPython.display import clear_output, display
from pathlib import Path
from utils.keypoints import Keypoint_obj

from jeremai.keypoints.predict import Predictor


PIL.Image.MAX_IMAGE_PIXELS = None

In [18]:
predictor = Predictor('keypoints.pkl')

In [19]:
df = pd.read_csv('data/train.csv')
ids = df[df['labels'].str.contains('fr-id1995-recto')]
files = [ path for path in ids['path']]

# ajouter les nouveaux documents dans le keypoints csv
keypointsDf = pd.read_csv('data/keypoints.csv')
keypointsList = []
def saveCSV():
    global keypointsList
    keypointsDf.to_csv('data/keypoints.csv', mode='w', index=False)
    keypointsList = keypointsDf.values

def setRow(filename, values):
    if (keypointsDf.loc[keypointsDf['path'] == filename].shape[0] > 0):
        keypointsDf.loc[keypointsDf['path'] == filename, ['a1', 'a2', 'a3', 'a4']] = values
    else:
        values.insert(0, filename)
        keypointsDf.loc[len(keypointsDf)] = values

for file in files:
    if (keypointsDf.loc[keypointsDf['path'] == file].shape[0] == 0):
        setRow(file, ['None','None','None','None'])

saveCSV()


In [20]:
canvasSize = 1000
canvas = MultiCanvas(2, width=canvasSize, height=canvasSize)
canvas[0].fill_style = '#584f4e'
canvas[0].fill_rect(0, 0, canvasSize, canvasSize)

keypoints = []

def handle_mouse_up(x, y):
    [o.set_selected(False) for o in keypoints if o.selected]
    
def handle_mouse_down(x, y):
    check_bool_pos = list(set([check_region.is_selected(x,y) for check_region in keypoints]))

def handle_mouse_move(x, y):    
    if [o for o in keypoints if o.selected]:
        with hold_canvas(canvas[1]):
            [o for o in keypoints if o.selected][-1].set_x_y(x,y)
            updateCanvas()

def updateCanvas():
    canvas[1].clear()
    [o.draw(canvas=canvas[1]) for o in keypoints]
    points = [(point.x, point.y) for point in keypoints]
    points.append ((keypoints[0].x, keypoints[0].y))
    canvas[1].stroke_lines(points)

canvas[1].on_mouse_down(handle_mouse_down)
canvas[1].on_mouse_up(handle_mouse_up)
canvas[1].on_mouse_move(handle_mouse_move)

In [21]:
current = 0
factor = 0
currentLabel = Label(value="0")
nameLabel = Label(value="hello")

def save(button):
    global factor, keypoints
    kps = [str(kp.x*factor) + " " + str(kp.y*factor) for kp in keypoints]
    setRow(keypointsList[current][0], kps)
    saveCSV()
    next(None)

def prev(button):
    global current
    current -= 1
    show()

def next(button):
    global current
    current += 1
    show()

def predict(button):
    global current, keypoints
    predictions = predictor.predict('data/train/'+keypointsList[current][0])
    kps = []
    cpt = 0
    for x, y in predictions:
        if(cpt == 0):
            kps += [Keypoint_obj(x/factor,y/factor, color='#f00')]
        else: 
            kps += [Keypoint_obj(x/factor,y/factor)]
        cpt+=1
    keypoints = kps
    updateCanvas()

def show():
    global factor, keypoints
    
    nameLabel.value = keypointsList[current][0]
    currentLabel.value = str(current + 1) + " / " + str(len(keypointsList))
    image = PIL.Image.open('data/train/' + keypointsList[current][0])
    image = ImageOps.exif_transpose(image)
    width, height = image.size
    ratio = height/width
    
    image = Image.from_file('data/train/' + keypointsList[current][0])
    canvas[0].clear()
    
    if(ratio < 1):
        #horizontal
        factor = width / canvasSize
        canvas[0].draw_image(image, 0, 0, width=canvasSize, height=canvasSize*ratio)
    else:
        #vertical
        factor = height / canvasSize
        canvas[0].draw_image(image, 0, 0, width=canvasSize/ratio, height=canvasSize)    

    row = keypointsList[current]
    kps = []
    if (row[1] != 'None'):
        for i in range(4):
            x, y = row[i+1].split()
            if(i == 0):
                kps += [Keypoint_obj(float(x)/factor,float(y)/factor, color='#f00')]
            else: 
                kps += [Keypoint_obj(float(x)/factor,float(y)/factor)]
        keypoints = kps
        updateCanvas()



keypoints = [Keypoint_obj(50,50, color='#f00'), Keypoint_obj(200,50), Keypoint_obj(200,150), Keypoint_obj(50,150)]   
    



prevButton = Button(
    description='Prev',
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)
prevButton.on_click(prev)

nextButton = Button(
    description='Next',
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)
nextButton.on_click(next)

predictButton = Button(
    description='Predict',
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
)
predictButton.on_click(predict)

saveButton = Button(
    description='Submit',
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
)
saveButton.on_click(save)

show()
updateCanvas()
buttons = VBox([currentLabel, nameLabel,prevButton, nextButton, predictButton, saveButton])
items = [canvas, buttons]
HBox(items)

In [100]:
keypointsDf

,path,a1,a2,a3,a4
0,17086d02-c337-4d5b-8459-8b5436cec3ca-1.png,1597.9890964604542 642.4195280304622,2239.2268106148895 765.3922249307153,2225.210685715339 989.6502233235232,1555.940721761803 954.6099110746471
1,4eda65887e70-cn3.jpg,1239.7614343975479 435.14610811351224,2096.857072090608 412.1059027991826,2106.07315421634 707.0205308226012,1235.1533933346816 716.2366129483331
2,cni2.jpg,1045.9978645287 558.108852212062,2748.3350342813114 561.3268808883429,2745.1170056050305 1153.4441573240338,1036.343778499857 1153.4441573240338
3,cni3.jpg,1298.489159388261 618.6429644697762,3399.179878922261 638.8031441198145,3391.1158070622455 1324.2492522211196,1294.4571234582534 1316.1851803611044
4,79fa6160-23e9-4144-ac97-6947acde07ed-1.png,1629.525377484443 348.412509169088,2284.7792165384285 355.42057161886333,2284.7792165384285 572.670507561896,1626.0213462595552 569.1664763370084
...,...,...,...,...,...
129,20110907_12140401.jpg,None,None,None,None
130,CNI_Fraude.jpg,None,None,None,None
131,Mr et Mme Bonne.jpg,None,None,None,None
132,4d004c9309424b5c8c2a8c253f3b9fea.jpg,None,None,None,None
